## 方案思路：33分类
label为还款日期距成交日期的天数，可能的情况有0天到31天，未还款定义为32，一共33个类别。
预测出每个label对应的概率，然后分别乘以应还的金额，就是每天需要还的金额。
线上分数：8500左右。
特征还有很多可以做，并且behavior表还没用，repay_logs表也还有很多有价值的东西没提取，因此分数还能提高。
主要问题：线下验证分数不靠谱，线上波动很大。
线下分类acc很低，需要找准特征工程的方向来提高分类准确率，此题当作分类任务和当作回归任务时的特征工程方向可能差别很大。

## Step1：数据预处理

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, log_loss, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse
from scipy.stats import kurtosis
import time
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [ ]:
# 解析'auditing_date', 'due_date', 'repay_date'列的值作为独立的日期列；
train_df = pd.read_csv('/Users/szkfzx/datasets/papadai/train.csv', parse_dates=['auditing_date', 'due_date', 'repay_date'])
# 还款日为空则填充为'due_date'
train_df['repay_date'] = train_df[['due_date', 'repay_date']].apply(
    lambda x: x['repay_date'] if x['repay_date'] != '\\N' else x['due_date'], axis=1
)
# 还款金额为空则填充为0
train_df['repay_amt'] = train_df['repay_amt'].apply(lambda x: x if x != '\\N' else 0).astype('float32')
# 标签为还款日和借款日的天数之差
train_df['label'] = (train_df['repay_date'] - train_df['auditing_date']).dt.days
# 还款金额为0，则标签为32
train_df.loc[train_df['repay_amt'] == 0, 'label'] = 32
# 分类器的标签值
clf_labels = train_df['label'].values
# 还款金额的标签值
amt_labels = train_df['repay_amt'].values
# 删除 分类器的标签值、还款金额的标签值、还款日期
del train_df['label'], train_df['repay_amt'], train_df['repay_date']
# 第一期应还的金额
train_due_amt_df = train_df[['due_amt']]
# 训练集的大小
train_num = train_df.shape[0]
train_df.head(100)

In [ ]:
# 解析'auditing_date', 'due_date'列的值作为独立的日期列
test_df = pd.read_csv('/Users/szkfzx/datasets/papadai/test.csv', parse_dates=['auditing_date', 'due_date'])
test_df.head(100)

In [ ]:
# 提交'listing_id', 'auditing_date', 'due_amt'
sub = test_df[['listing_id', 'auditing_date', 'due_amt']]
sub.head(100)

In [ ]:
# 拼接训练集和测试集
df = pd.concat([train_df, test_df], axis=0, ignore_index=True)
df.head(100)

In [ ]:
listing_info_df = pd.read_csv('/Users/szkfzx/datasets/papadai/listing_info.csv')
# 删除标的属性表的'user_id' 'auditing_date'
del listing_info_df['user_id'], listing_info_df['auditing_date']
# 合并训练集和测试集、标的属性表
df = df.merge(listing_info_df, on='listing_id', how='left')
df.head(100)

In [ ]:
# 解析'reg_mon', 'insertdate'列的值作为独立的日期列
user_info_df = pd.read_csv('/Users/szkfzx/datasets/papadai/user_info.csv', parse_dates=['reg_mon', 'insertdate'])
# 重命名'insertdate'为'info_insert_date'
user_info_df.rename(columns={'insertdate': 'info_insert_date'}, inplace=True)
# 表中有少数user不止一条记录，因此按日期'info_insert_date'排序，去重，只保留最新的一条记录。
user_info_df = user_info_df.sort_values(by='info_insert_date', ascending=False).drop_duplicates('user_id').reset_index(drop=True)
# 合并训练集和测试集、标的属性表、用户信息表
df = df.merge(user_info_df, on='user_id', how='left')
df.head(100)

In [ ]:
# 解析‘insertdate'列的值作为独立的日期列
user_tag_df = pd.read_csv('/Users/szkfzx/datasets/papadai/user_taglist.csv', parse_dates=['insertdate'])
# 重命名'insertdate'为'info_insert_date'
user_tag_df.rename(columns={'insertdate': 'tag_insert_date'}, inplace=True)
# 表中有少数user不止一条记录，因此按日期'tag_insert_date'排序，去重，只保留最新的一条记录。
user_tag_df = user_tag_df.sort_values(by='tag_insert_date', ascending=False).drop_duplicates('user_id').reset_index(drop=True)
# 合并训练集和测试集、标的属性表、用户信息表、 用户画像标签列表
df = df.merge(user_tag_df, on='user_id', how='left')
df.head(100)

In [ ]:
# 历史记录表能做的特征远不止这些
# 解析'due_date', 'repay_date'列的值作为独立的日期列
repay_log_df = pd.read_csv('/Users/szkfzx/datasets/papadai/user_repay_logs.csv', parse_dates=['due_date', 'repay_date'])
# 由于题目任务只预测第一期的还款情况，因此这里只保留第一期的历史记录。当然非第一期的记录也能提取很多特征。
repay_log_df = repay_log_df[repay_log_df['order_id'] == 1].reset_index(drop=True)
# 还款'repay'为1，逾期'repay'为0
repay_log_df['repay'] = repay_log_df['repay_date'].astype('str').apply(lambda x: 1 if x != '2200-01-01' else 0)
# 还款日提前的天数
repay_log_df['early_repay_days'] = (repay_log_df['due_date'] - repay_log_df['repay_date']).dt.days
# 还款日提前的天数为负，则设为-1
repay_log_df['early_repay_days'] = repay_log_df['early_repay_days'].apply(lambda x: x if x >= 0 else -1)
repay_log_df.head(100)

In [ ]:
# 删除'listing_id', 'order_id', 'due_date', 'repay_date', 'repay_amt'
for f in ['listing_id', 'order_id', 'due_date', 'repay_date', 'repay_amt']:
    del repay_log_df[f]
# 按照'user_id'分组
group = repay_log_df.groupby('user_id', as_index=False)
repay_log_df.head(100)

In [ ]:
group.head(100)

In [ ]:
repay_log_df = repay_log_df.merge(
    group['repay'].agg({'repay_mean': 'mean'}), on='user_id', how='left'
)
repay_log_df.head(100)

In [ ]:
repay_log_df = repay_log_df.merge(
    group['early_repay_days'].agg({
        'early_repay_days_max': 'max', 'early_repay_days_median': 'median', 'early_repay_days_sum': 'sum',
        'early_repay_days_mean': 'mean', 'early_repay_days_std': 'std'
    }), on='user_id', how='left'
)
repay_log_df.head(100)

In [ ]:
repay_log_df = repay_log_df.merge(
    group['due_amt'].agg({
        'due_amt_max': 'max', 'due_amt_min': 'min', 'due_amt_median': 'median',
        'due_amt_mean': 'mean', 'due_amt_sum': 'sum', 'due_amt_std': 'std',
        'due_amt_skew': 'skew', 'due_amt_kurt': kurtosis, 'due_amt_ptp': np.ptp
    }), on='user_id', how='left'
)
repay_log_df.head(100)

In [ ]:
# 删除用户还款日志表'repay' 'early_repay_days' 'due_amt'
del repay_log_df['repay'], repay_log_df['early_repay_days'], repay_log_df['due_amt']
# 按'user_id'去重
repay_log_df = repay_log_df.drop_duplicates('user_id').reset_index(drop=True)
# 合并训练集和测试集、标的属性表、用户信息表、 用户画像标签列表、用户还款日志表
df = df.merge(repay_log_df, on='user_id', how='left')

df.head(100)

In [ ]:
# 映射'gender', 'cell_province', 'id_province', 'id_city'
cate_cols = ['gender', 'cell_province', 'id_province', 'id_city']
# unique()返回参数数组中所有不同的值,并按照从小到大排序
# nunique()获取列中所有唯一值的数量
# zip之后默认的都会转化成tuple格式的，如果要还原会原来的格式需要借助于map函数
# dict转为pandas中的dataframe格式,并将dict的keys作为数据列,且赋予该数据列新的列名
for f in cate_cols:
    df[f] = df[f].map(dict(zip(df[f].unique(), range(df[f].nunique())))).astype('int32')
# 计算每日还款数
df['due_amt_per_days'] = df['due_amt'] / (train_df['due_date'] - train_df['auditing_date']).dt.days

df.head(100)

In [ ]:
date_cols = ['auditing_date', 'due_date', 'reg_mon', 'info_insert_date', 'tag_insert_date']
for f in date_cols:
    # 'reg_mon', 'info_insert_date', 'tag_insert_date'取【年】
    if f in ['reg_mon', 'info_insert_date', 'tag_insert_date']:
        df[f + '_year'] = df[f].dt.year
    #'auditing_date', 'due_date', 'reg_mon', 'info_insert_date', 'tag_insert_date'取【月】
    df[f + '_month'] = df[f].dt.month
    # 'auditing_date', 'due_date', 'info_insert_date', 'tag_insert_date'取【日】【周几】
    if f in ['auditing_date', 'due_date', 'info_insert_date', 'tag_insert_date']:
        df[f + '_day'] = df[f].dt.day
        df[f + '_dayofweek'] = df[f].dt.dayofweek
#删除'auditing_date', 'due_date', 'reg_mon', 'info_insert_date', 'tag_insert_date'
df.drop(columns=date_cols, axis=1, inplace=True)

df.head(100)

In [ ]:
#处理标签，以'|'分割
df['taglist'] = df['taglist'].astype('str').apply(lambda x: x.strip().replace('|', ' ').strip())
# CountVectorizer：只考虑词汇在文本中出现的频率
# TfidfVectorizer：除了考量某词汇在文本出现的频率，还关注包含这个词汇的所有文本的数量能够削减
# 高频没有意义的词汇出现带来的影响, 挖掘更有意义的特征
tag_cv = CountVectorizer(min_df=10, max_df=0.9).fit_transform(df['taglist'])
# 删除'user_id' 'listing_id' 'taglist'
del df['user_id'], df['listing_id'], df['taglist']
#one-hot编码
df = pd.get_dummies(df, columns=cate_cols)
# 横向合并df和稀疏后的features,并存储为csr格式
df = sparse.hstack((df.values, tag_cv), format='csr', dtype='float32')
# 分割训练集和测试集
train_values, test_values = df[:train_num], df[train_num:]

In [ ]:
print(train_values.shape)
print(test_values.shape)

## Step2：模型训练

In [ ]:
# 五折验证也可以改成一次验证，按时间划分训练集和验证集，以避免由于时序引起的数据穿越问题。
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2019)
# LGBMClassifier分类器
clf = LGBMClassifier(
    learning_rate=0.05,
    n_estimators=10000,
    subsample=0.8,
    subsample_freq=1,
    colsample_bytree=0.8,
    random_state=2019
)
# 训练输出金额初始化
amt_oof = np.zeros(train_num)
# 训练输出还款日期预测初始化
prob_oof = np.zeros((train_num, 33))
# 测试输出还款日期预测初始化
test_pred_prob = np.zeros((test_values.shape[0], 33))
for i, (trn_idx, val_idx) in enumerate(skf.split(train_values, clf_labels)):
    print(i, 'fold...')
    t = time.time()

    #训练集X y
    trn_x, trn_y = train_values[trn_idx], clf_labels[trn_idx]
    #验证集X y
    val_x, val_y = train_values[val_idx], clf_labels[val_idx]
    #验证集还款金额
    val_repay_amt = amt_labels[val_idx]
    #验证集应还金额
    val_due_amt = train_due_amt_df.iloc[val_idx]

    #训练模型
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        early_stopping_rounds=100, verbose=5
    )
    # shape = (-1, 33)
    # 验证集每日预测
    val_pred_prob_everyday = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    # 预测矩阵
    prob_oof[val_idx] = val_pred_prob_everyday
    # 验证集今日预测
    val_pred_prob_today = [val_pred_prob_everyday[i][val_y[i]] for i in range(val_pred_prob_everyday.shape[0])]
    # 验证集还款金额预测
    val_pred_repay_amt = val_due_amt['due_amt'].values * val_pred_prob_today
    print('val rmse:', np.sqrt(mean_squared_error(val_repay_amt, val_pred_repay_amt)))
    print('val mae:', mean_absolute_error(val_repay_amt, val_pred_repay_amt))
    amt_oof[val_idx] = val_pred_repay_amt
    # 测试集预测
    test_pred_prob += clf.predict_proba(test_values, num_iteration=clf.best_iteration_) / skf.n_splits

    print('runtime: {}\n'.format(time.time() - t))

print('\ncv rmse:', np.sqrt(mean_squared_error(amt_labels, amt_oof)))
print('cv mae:', mean_absolute_error(amt_labels, amt_oof))
print('cv logloss:', log_loss(clf_labels, prob_oof))
print('cv acc:', accuracy_score(clf_labels, np.argmax(prob_oof, axis=1)))

In [ ]:
prob_cols = ['prob_{}'.format(i) for i in range(33)]
for i, f in enumerate(prob_cols):
    sub[f] = test_pred_prob[:, i]

## Step3：提交结果

In [ ]:
sub_example = pd.read_csv('/Users/szkfzx/datasets/papadai/submission.csv', parse_dates=['repay_date'])
sub_example = sub_example.merge(sub, on='listing_id', how='left')
sub_example['days'] = (sub_example['repay_date'] - sub_example['auditing_date']).dt.days
# shape = (-1, 33)
test_prob = sub_example[prob_cols].values
test_labels = sub_example['days'].values
test_prob = [test_prob[i][test_labels[i]] for i in range(test_prob.shape[0])]
sub_example['repay_amt'] = sub_example['due_amt'] * test_prob
sub_example[['listing_id', 'repay_date', 'repay_amt']].to_csv('sub.csv', index=False)